In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
import torchaudio
import numpy as np
from tqdm import tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

Using cpu


In [ ]:
epochs = 50
lr = 0.001
batch_size = 6

In [ ]:
# Make all tensor in a batch the same length by padding with zeros

def pad_sequence(batch):
    batch = [item.t() for item in batch]
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
    return batch.permute(0, 2, 1)


def collate_fn(batch):
    tensors, targets = [], []

    for waveform, label in batch:
        tensors += [waveform]
        targets += torch.tensor([label])

    tensors = pad_sequence(tensors)
    targets = torch.stack(targets)

    return tensors, targets

In [ ]:
class AudioDataset(Dataset):
    def __init__(self, dir_path):
        self.dir_path = dir_path
        self.classes = os.listdir(self.dir_path)
        self.data_paths = []
        self.labels = []

        for root, dirs, files in os.walk(self.dir_path):
            for file in files:
                label = os.path.basename(root)
                data_path = os.path.join(root, file)
                self.data_paths.append(data_path)
                self.labels.append(self.classes.index(label))

        print("classes: ", self.classes)
        print(f"{len(self.labels)} datas loaded from {len(set(self.labels))} classes")

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        data_path = self.data_paths[index]
        label = self.labels[index]
        signal, sample_rate = torchaudio.load(data_path)
        signal_mono = torch.mean(signal, dim=0, keepdim=True)

        new_sample_rate = 8000
        transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)
        signal_mono_transformed = transform(signal_mono)

        return signal_mono_transformed, label

In [ ]:
ROOT = '/content/drive/MyDrive/Dataset/audio_classification/'
dataset = AudioDataset(ROOT + "dataset")

train_size = int(len(dataset)*0.9)
test_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=True, 
                                                collate_fn=collate_fn,
                                                num_workers=num_workers, 
                                                pin_memory=pin_memory)

test_data_loader = torch.utils.data.DataLoader(test_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=False,
                                                collate_fn=collate_fn,
                                                num_workers=num_workers, 
                                                pin_memory=pin_memory)

classes:  ['zahra', 'sajjad', 'benyamin', 'alireza', 'maryam', 'nahid', 'parisa', 'mohammadali', 'morteza', 'zeynab']
243 datas loaded from 10 classes


In [ ]:
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=10, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=60, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = F.softmax(x, dim=1)
        return x

    def accuracy(self, preds, labels):
        maxs, indices = torch.max(preds, 1)
        acc = torch.sum(indices == labels) / len(preds)
        return acc.cpu()

In [ ]:
model = M5(n_output=10).to(device)
print(model)

# count_parameters
n = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters: %s" % n)

M5(
  (conv1): Conv1d(1, 32, kernel_size=(60,), stride=(10,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=64, out_features=10, bias=True)
)
Numbe

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()

In [ ]:
# train
model.train()

for epoch in range(epochs):
    train_loss = 0.0
    train_acc = 0.0
    for audios, labels in tqdm(train_data_loader):
        audios, labels = audios.to(device), labels.to(device)
        labels_one_hot = torch.nn.functional.one_hot(labels, num_classes=10).type(torch.FloatTensor).to(device)

        preds = model(audios)
        loss = loss_function(preds, labels_one_hot)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss
        train_acc += model.accuracy(preds, labels)
    
    total_loss = train_loss / len(train_data_loader)
    total_acc = train_acc / len(train_data_loader)

    print(f"Epoch: {epoch}, Loss: {total_loss}, Acc: {total_acc}")

100%|██████████| 37/37 [00:44<00:00,  1.19s/it]


Epoch: 0, Loss: 2.259014129638672, Acc: 0.20720718801021576


100%|██████████| 37/37 [00:02<00:00, 17.00it/s]


Epoch: 1, Loss: 2.115438222885132, Acc: 0.4324324429035187


100%|██████████| 37/37 [00:02<00:00, 16.98it/s]


...

Epoch: 47, Loss: 1.4917360544204712, Acc: 0.990990936756134


100%|██████████| 37/37 [00:02<00:00, 17.12it/s]


Epoch: 48, Loss: 1.5346221923828125, Acc: 0.9684685468673706


100%|██████████| 37/37 [00:02<00:00, 17.11it/s]

Epoch: 49, Loss: 1.5114223957061768, Acc: 0.9729729890823364


In [ ]:
model.eval()

test_loss = 0.0
test_acc = 0.0
for audios, labels in tqdm(test_data_loader):
    audios, labels = audios.to(device), labels.to(device)
    labels_one_hot = torch.nn.functional.one_hot(labels, num_classes=10).type(torch.FloatTensor).to(device)

    preds = model(audios)
    loss = loss_function(preds, labels_one_hot)

    test_loss += loss
    test_acc += model.accuracy(preds, labels)

total_loss = test_loss / len(test_data_loader)
total_acc = test_acc / len(test_data_loader)

print(f"Loss: {total_loss}, Acc: {total_acc}")

100%|██████████| 5/5 [00:04<00:00,  1.04it/s]

Loss: 1.4841755628585815, Acc: 1.0


In [ ]:
torch.save(model.state_dict(), ROOT + "weights.pth")

In [ ]:
# Inference

signal, sample_rate = torchaudio.load("input/test.ogg")

# preprocess
signal = torch.mean(signal, dim=0, keepdim=True)
new_sample_rate = 8000
transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)
signal = transform(signal)
signal = signal[:, 32000:40000]
signal = signal.unsqueeze(0).to(device)

# process
preds = model(signal)

# postprocess
preds = preds.cpu().detach().numpy()
output = np.argmax(preds)
print(output)

### Telegram BOT

In [ ]:
!pip install pyTelegramBotAPI

In [24]:
import telebot
import torch
import torchaudio


Token = open(ROOT + "token.txt","r").read()
bot = telebot.TeleBot(Token)
data = ['alireza', 'benyamin', 'maryam', 'mohammadali', 'morteza', 'nahid', 'parisa', 'sajjad', 'zahra', 'zeynab']

@bot.message_handler(commands=['start'])
def start(messages):
    bot.send_message(messages.chat.id, f'Hey {messages.from_user.first_name}.\ngive me a voice')

@bot.message_handler(content_types=['voice'])
def voice(message):
  try:
    audio_info = bot.get_file(message.voice.file_id)
    
    downloaded_file = bot.download_file(audio_info.file_path)
    src = audio_info.file_path
    with open(src, 'wb') as audio_file:
        audio_file.write(downloaded_file)

    signal, sample_rate = torchaudio.load(src)

    # preprocess
    signal = torch.mean(signal, dim=0, keepdim=True)
    transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=8000)
    signal = transform(signal)
    voice = signal.unsqueeze(0).to(device)

    # process
    preds = model(voice)

    # postprocess
    preds = preds.cpu().detach().numpy()
    output = np.argmax(preds)
    if data[output]:
        bot.reply_to(message, data[output])
    else:
      print("Sorry stranger we dont serve here.")
  except Exception as e:
    bot.reply_to(message, f'Something went wrong.\nCall my dad @Benyamin_Zojaji\nError log:{e}')
  
@bot.message_handler(commands=['help'])
def help(message):
  bot.reply_to(message, f'Give me a voice, Ill tell you who you are\nMade with 💖 in the middle of University exams👨‍💻\n@Benyamin_Zojaji')

bot.polling()